In [2]:
!pip install torch

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [4]:
data1 = pd.read_csv('Webpages_Classification_train_data.csv')

In [5]:
data1.head()

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,0,http://members.tripod.com/russiastation/,40,42.77.221.155,Taiwan,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good
1,1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,United States,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good
2,2,http://www.naef-usa.com/,24,24.232.54.41,Argentina,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good
3,3,http://www.ff-b2b.de/,21,147.22.38.45,United States,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad
4,4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,United States,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good


In [6]:
data1.dtypes

Unnamed: 0      int64
url            object
url_len         int64
ip_add         object
geo_loc        object
tld            object
who_is         object
https          object
js_len        float64
js_obf_len    float64
content        object
label          object
dtype: object

In [7]:
bads = data1.loc[data1["label"] == "bad"]

In [8]:
bads.head()

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
3,3,http://www.ff-b2b.de/,21,147.22.38.45,United States,de,incomplete,no,720.0,532.800,fire cumshot sodomize footaction tortur failed...,bad
64,64,http://dracula-land.sexomultiple.com/cg-amateu...,63,97.83.163.187,United States,com,incomplete,no,621.0,378.810,dingleberry suckmyass transexual waysted piker...,bad
114,114,http://www.angelfire.com/darkside/ms_sassy/,43,76.72.209.223,United States,com,incomplete,no,513.0,338.580,niggarding masterblaster buttplug bigass gangs...,bad
144,144,http://www.sexystat.com,23,130.5.218.123,United States,com,incomplete,no,420.3,193.338,attack loverocket bulldike cumshot groe slant ...,bad
167,167,http://www.bizarre-sex.ws/,26,49.126.108.27,Nepal,ws,incomplete,no,335.7,0.000,muffindiver husky peepshow magicwand mad phukk...,bad


In [9]:
bads.groupby('label').size()

label
bad    27253
dtype: int64

In [10]:
goods = data1.loc[data1["label"] == "good"]

In [11]:
goods.head()

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,0,http://members.tripod.com/russiastation/,40,42.77.221.155,Taiwan,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good
1,1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,United States,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good
2,2,http://www.naef-usa.com/,24,24.232.54.41,Argentina,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good
4,4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,United States,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good
5,5,http://efilmcritic.com/hbs.cgi?movie=311,40,8.28.167.23,United States,com,complete,yes,39.5,0.0,Signals. \n territorial jurisdictions. common....,good


In [12]:
goods.groupby('label').size()

label
good    1172747
dtype: int64

In [13]:
bads = bads.iloc[0:1001, 0:12]

In [14]:
goods = goods.iloc[0:2001, 0:12]

In [15]:
bads.groupby('label').size()

label
bad    1001
dtype: int64

In [16]:
goods.groupby('label').size()

label
good    2001
dtype: int64

In [17]:
data = pd.concat([bads,goods])

In [18]:
data.head()

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
3,3,http://www.ff-b2b.de/,21,147.22.38.45,United States,de,incomplete,no,720.0,532.800,fire cumshot sodomize footaction tortur failed...,bad
64,64,http://dracula-land.sexomultiple.com/cg-amateu...,63,97.83.163.187,United States,com,incomplete,no,621.0,378.810,dingleberry suckmyass transexual waysted piker...,bad
114,114,http://www.angelfire.com/darkside/ms_sassy/,43,76.72.209.223,United States,com,incomplete,no,513.0,338.580,niggarding masterblaster buttplug bigass gangs...,bad
144,144,http://www.sexystat.com,23,130.5.218.123,United States,com,incomplete,no,420.3,193.338,attack loverocket bulldike cumshot groe slant ...,bad
167,167,http://www.bizarre-sex.ws/,26,49.126.108.27,Nepal,ws,incomplete,no,335.7,0.000,muffindiver husky peepshow magicwand mad phukk...,bad


In [19]:
data.groupby('label').size()

label
bad     1001
good    2001
dtype: int64

In [20]:
from sklearn.utils import shuffle
data = shuffle(data)

In [21]:
data.head()

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
656,656,http://www.sykospark.net/goth/,30,101.237.72.212,China,net,incomplete,yes,184.0,0.0,"In namibia tales, e.g. the ugly duckling, the ...",good
967,967,http://www.charlestonwomensmedicalcenter.com/,45,70.80.230.64,Canada,com,complete,yes,39.0,0.0,Universe does and stars has been to. Are regis...,good
104,104,http://www.mykoweb.com/caf/species/hypholoma_f...,61,180.239.193.61,South Korea,com,complete,no,92.0,0.0,arabs skinflute nigerian jiggy gipp bitchslap ...,good
6461,6461,http://www.geocities.com/hollywood/mansion/204...,58,126.176.181.69,Japan,com,incomplete,no,657.0,459.9,Locale specific libraries.) poortinga.. De min...,bad
859,859,http://www.popswine.com/,24,168.53.36.166,United States,com,complete,yes,145.0,0.0,Leeuwenhoek in the japan ground self-defense f...,good


In [50]:
df_train = data.iloc[0:2001, 0:12]
df_test = data.iloc[2001:3001, 0:12]
df_test.dtypes

Unnamed: 0      int64
url            object
url_len         int64
ip_add         object
geo_loc        object
tld            object
who_is         object
https          object
js_len        float64
js_obf_len    float64
content        object
label          object
dtype: object

In [51]:
class config:
    BATCH_SIZE = 128
    DEVICE =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    LEARNING_RATE = 2e-5
    EPOCHS = 20

In [52]:
# Making the custom dataset for pytorch
class MaliciousBenignData(Dataset):
    def __init__(self, df):
        self.df = df
        self.input = self.df.drop(columns = ['label']).values
        self.target = self.df.label
        
    def __len__(self):
        return (len(self.df))
    
    def __getitem__(self, idx):
        return (torch.tensor(self.input[idx]), torch.tensor(self.target[idx]))

In [53]:
# Creating the dataloader for pytorch
def create_dataloader(df, batch_size):
    cls = MaliciousBenignData(df)
    return DataLoader(
        cls,
        batch_size = batch_size,
        num_workers = 0
    )

df_train_loader = create_dataloader(df_train, batch_size = config.BATCH_SIZE)
df_test_loader = create_dataloader(df_test, batch_size = 1) # Here for testing using the batch size as 1



In [54]:
df_train_loader.__dict__

{'dataset': <__main__.MaliciousBenignData at 0x1b4a5885250>,
 'num_workers': 0,
 'prefetch_factor': 2,
 'pin_memory': False,
 'timeout': 0,
 'worker_init_fn': None,
 '_DataLoader__multiprocessing_context': None,
 '_dataset_kind': 0,
 'batch_size': 128,
 'drop_last': False,
 'sampler': <torch.utils.data.sampler.SequentialSampler at 0x1b4a5885520>,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x1b4a58857c0>,
 'generator': None,
 'collate_fn': <function torch.utils.data._utils.collate.default_collate(batch)>,
 'persistent_workers': False,
 '_DataLoader__initialized': True,
 '_IterableDataset_len_called': None,
 '_iterator': None}

In [55]:
class dnn(nn.Module):
    def __init__(self):
        super(dnn, self).__init__()

        self.fc1 = nn.Linear(10, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 128)
        self.out = nn.Linear(128, 1)

        self.dropout1 = nn.Dropout(p = 0.2)        
        self.dropout2 = nn.Dropout(p = 0.3)
        self.batchn1 = nn.BatchNorm1d(num_features = 64)
        self.batchn2 = nn.BatchNorm1d(num_features = 128)

    def forward(self, inputs):

        t = self.fc1(inputs)
        t = F.relu(t)
        t = self.batchn1(t)
        t = self.dropout1(t)
        t = self.fc2(t)
        t = F.relu(t)
        t = self.batchn2(t)
        t = self.dropout2(t)
        t = self.fc3(t)
        t = F.relu(t)
        t = self.out(t)

        return t

In [56]:
model = dnn()
model.to(config.DEVICE)
print (model)

dnn(
  (fc1): Linear(in_features=10, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (batchn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [57]:
# Criterian and the Optimizer for the model
criterian = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = config.LEARNING_RATE)

In [58]:
# Simple Binary Accuracy Function
def binary_acc(predictions, y_test):
    y_pred = torch.round(torch.sigmoid(predictions))
    correct = (y_pred == y_test).sum().float()
    acc = torch.round((correct/y_test.shape[0])*100)
    return acc

In [59]:
# Training function

def train_model(model, device, data_loader, optimizer, criterian):
    # Putting the model in training mode
    model.train()

    for epoch in range(1, config.EPOCHS+1):
        epoch_loss = 0
        epoch_acc = 0
        for X, y in data_loader:

            X = X.to(device)
            y_ = torch.tensor(y.unsqueeze(1), dtype = torch.float32)
            y = y_.to(device)

            # Zeroing the gradient
            optimizer.zero_grad()

            predictions = model(X.float())

            loss = criterian(predictions, y)
            acc = binary_acc(predictions, y)

            loss.backward() # Calculate Gradient
            optimizer.step() # Updating Weights

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        print (f"Epoch -- {epoch} | Loss : {epoch_loss/len(data_loader): .5f} | Accuracy : {epoch_acc/len(data_loader): .5f}")

In [60]:
# Evaluation Function

def eval_model(model, device, data_loader):
    # Putting the model in evaluation mode
    model.eval()

    y_pred = []
    y_test_al = []

    with torch.no_grad():
        for X_test, y_test in data_loader:
            X_test = X_test.to(device)

            predictions = model(X_test.float())
            pred = torch.round(torch.sigmoid(predictions))

            y_test_al.append(y_test.tolist())
            y_pred.append(pred.tolist())

        # Changing the Predictions into list 
        y_test_al = [ele[0] for ele in y_test_al]
        y_pred = [int(ele[0][0]) for ele in y_pred] # the format of the prediction is [[[0]], [[1]]]

        return (y_test_al, y_pred)

In [61]:
train_model(model, config.DEVICE, df_train_loader, optimizer, criterian)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.